In [113]:
from __future__ import annotations
from typing import Callable
from dataclasses import dataclass

from nptyping import NDArray, Float, Shape
import numpy as np

rng = np.random.default_rng()
FloatVec = NDArray[Shape['*'], Float]

class Activation:
    @staticmethod
    def relu(X: FloatVec) -> FloatVec:
        return X * (X > 0)

    @staticmethod
    def softmax(X: FloatVec) -> FloatVec:
        Y = np.exp(X)
        return Y / Y.sum()

@dataclass
class InputLayer:
    width: int

@dataclass
class Layer:
    activation: Callable[[FloatVec], FloatVec]
    width: int

    def __repr__(self) -> str:
        return f'Layer({self.activation.__name__}, width={self.width})'
    
    def __call__(self, input: FloatVec, weights: NDArray[Shape['Width,Width'], Float]) -> FloatVec:
        """Predict outputs for input from previous layer, based on weights with previous layer"""
        # 1 appended to input to account for bias (present in last column in weights matrix)
        return self.activation(weights @ np.append(input, 1))  

    def to_dict(self):
        return { "activation": self.activation.__name__, "width": self.width }

    @staticmethod
    def from_dict(layer_dict) -> Layer:
        return Layer(
            activation=getattr(Activation, layer_dict['activation']),
            width=layer_dict['width']
        )

@dataclass
class Model:
    """Feed-Forward Neural Network"""
    input_width: int
    layers: list[Layer]
    weights: list[NDArray[Shape['Width,Width'], Float]] | None = None

    def __post_init__(self) -> None:
        """Initialize weight matrices (including biases) between each layers"""
        if self.weights is None:
            layers = [InputLayer(width=self.input_width)] + self.layers
            self.weights = [
                # weight matrix of layer with its previous layer
                rng.random((curr_layer.width, prev_layer.width+1))       # +1 to account for biases
                for prev_layer, curr_layer in zip(layers[:-1], layers[1:])
            ]
    
    def __eq__(self, model: Model) -> bool:
        return (
            self.input_width == model.input_width and
            self.layers == model.layers and
            all(np.array_equal(w1,w2) for w1,w2 in zip(self.weights, model.weights))
        )

    def __call__(self, input: FloatVec) -> FloatVec:
        """Predict outputs for input"""
        for layer, weights in zip(self.layers, self.weights):
            input = layer(input, weights)
        return input

    @property
    def depth(self) -> int:
        return len(self.layers)

    def to_dict(self):
        return {
            'input_width': self.input_width,
            'layers': [layer.to_dict() for layer in self.layers],
            'weights': [weights.tolist() for weights in self.weights]
        }
    
    @staticmethod
    def from_dict(model_dict) -> Model:
        return Model(
            input_width=model_dict['input_width'],
            layers=[Layer.from_dict(layer_dict) for layer_dict in model_dict['layers']],
            weights=[np.asarray(weights_list) for weights_list in model_dict['weights']],
        )


layers = [
    Layer(Activation.relu, width=10),
    Layer(Activation.relu, width=10),
    Layer(Activation.softmax, width=3)      # output layer
]
model = Model(input_width=4, layers=layers)

In [66]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import pandas as pd
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['class'] = [iris.target_names[i] for i in iris.target]
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [114]:
model(X_train[0])

array([9.99999981e-01, 1.87191670e-08, 1.82168040e-13])

In [ ]:
d = model.to_dict()

In [ ]:
m = Model.from_dict(d) 

In [111]:
model == m

True